In [1]:
#-*- coding:utf-8 -*-
from diffusion_model.trainer import GaussianDiffusion, num_to_groups
from diffusion_model.trainer import GaussianDiffusion, Trainer
from diffusion_model.unet import create_model
from torchvision.transforms import Compose, Lambda
from utils.dtypes import LabelEnum
import nibabel as nib
import torchio as tio
import numpy as np
import argparse
import torch
import os
import glob

2025-06-10 09:22:07.810756: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-10 09:22:07.825393: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-10 09:22:07.830047: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-10 09:22:07.844557: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-10 09:22:08.586993: W tensorflow/compiler/tf2

APEX: OFF


In [2]:

exportfolder ='../../../result/generator/Normal_SWI'
inputfolder = '../../../data/stroke_swi_nii/Skyra&Verio/masks'
input_size = 128
depth_size = 64
batchsize = 8
weightfile = '../../../model/med_ddpm/swi/model-4520.pt'
num_channels = 64
num_res_blocks = 1
num_samples = 1
in_channels =3
out_channels = 1
device = "cuda:1" if torch.cuda.is_available() else "cpu"
print("Device: ", device)

mask_list = sorted(glob.glob(f"{inputfolder}/*.nii.gz"))
print(len(mask_list))


Device:  cuda:1
316


In [3]:

def resize_img_4d(input_img):
    d, h, w, c = input_img.shape  # Axial 기준: (D, H, W, C)
    result_img = np.zeros((depth_size, input_size, input_size, in_channels - 1))  # (D, H, W, C)

    if d != depth_size or h != input_size or w != input_size:
        for ch in range(c):
            buff = input_img[..., ch]  # (D, H, W)
            img = tio.ScalarImage(tensor=buff[np.newaxis, ...])  # (1, D, H, W)
            cop = tio.Resize((depth_size, input_size, input_size))
            img = np.asarray(cop(img))[0]  # (D, H, W)
            result_img[..., ch] = img
        return result_img
    else:
        return input_img


def label2masks(masked_img):
    result_img = np.zeros(masked_img.shape + (in_channels-1,))
    result_img[masked_img==LabelEnum.BRAINAREA.value, 0] = 1
    result_img[masked_img==LabelEnum.TUMORAREA.value, 1] = 1
    return result_img


input_transform = Compose([
    Lambda(lambda t: torch.tensor(t).float()),
    Lambda(lambda t: (t * 2) - 1),
    Lambda(lambda t: t.permute(3, 0, 1, 2)),
    Lambda(lambda t: t.unsqueeze(0))
])

model = create_model(input_size, num_channels, num_res_blocks, in_channels=in_channels, out_channels=out_channels).to(device)


diffusion = GaussianDiffusion(
    model,
    image_size = input_size,
    depth_size = depth_size,
    timesteps = 250,   # number of steps
    loss_type = 'L1', 
    with_condition=True,
).to(device)
diffusion.load_state_dict(torch.load(weightfile,map_location=device))
print("Model Loaded!")


Model Loaded!


In [4]:
max_file_size_kb = 3450
max_retry = 10
img_dir = exportfolder
for k, inputfile in enumerate(mask_list):
    left = len(mask_list) - (k + 1)
    print("LEFT: ", left)

    ref = nib.load(inputfile)
    msk_name = inputfile.split('/')[-1]
    refImg = ref.get_fdata()
    img = label2masks(refImg)
    img = resize_img_4d(img)
    input_tensor = input_transform(img)
    condition_tensor = input_tensor.to(device)

    for sample_idx in range(num_samples):
        saved_count = 0
        retry = 0
        file_saved = False
        generated = diffusion.sample(batch_size=batchsize, condition_tensors=condition_tensor.repeat(batchsize, 1, 1, 1, 1))
        generated = generated.unsqueeze(1).cpu().numpy()  # (B, 1, D, H, W)

        for b in range(batchsize):
            sampleImage = generated[b][0]  # shape: (D, H, W)
            sampleImage = sampleImage.reshape(refImg.shape)

            # 저장 경로 생성
            out_name = f"{b}_{msk_name}"
            nifti_path = os.path.join(img_dir, out_name)
            nifti_img = nib.Nifti1Image(sampleImage, affine=ref.affine)
            nib.save(nifti_img, nifti_path)
            file_size_kb = os.path.getsize(nifti_path) / 1024
            # if file_size_kb <= max_file_size_kb:
            #     file_saved = True
            #     saved_count += 1
            #     break
            # else:
            #     os.remove(nifti_path)  # 너무 크면 삭제하고 재시도
            #     continue
    
    torch.cuda.empty_cache()

LEFT:  315
LEFT:  314
LEFT:  313
LEFT:  312
LEFT:  311
LEFT:  310
LEFT:  309
LEFT:  308
LEFT:  307
LEFT:  306
LEFT:  305
LEFT:  304
LEFT:  303
LEFT:  302
LEFT:  301
LEFT:  300
LEFT:  299
LEFT:  298
LEFT:  297
LEFT:  296
LEFT:  295
LEFT:  294
LEFT:  293
LEFT:  292
LEFT:  291
LEFT:  290
LEFT:  289
LEFT:  288
LEFT:  287
LEFT:  286
LEFT:  285
LEFT:  284
LEFT:  283
LEFT:  282
LEFT:  281
LEFT:  280
LEFT:  279
LEFT:  278
LEFT:  277
LEFT:  276
LEFT:  275
LEFT:  274
LEFT:  273
LEFT:  272
LEFT:  271
LEFT:  270
LEFT:  269
LEFT:  268
LEFT:  267
LEFT:  266
LEFT:  265
LEFT:  264
LEFT:  263
LEFT:  262
LEFT:  261
LEFT:  260
LEFT:  259
LEFT:  258
LEFT:  257
LEFT:  256
LEFT:  255
LEFT:  254
LEFT:  253
LEFT:  252
LEFT:  251
LEFT:  250
LEFT:  249
LEFT:  248
LEFT:  247
LEFT:  246
LEFT:  245
LEFT:  244
LEFT:  243
LEFT:  242
LEFT:  241
LEFT:  240
LEFT:  239
LEFT:  238
LEFT:  237
LEFT:  236
LEFT:  235
LEFT:  234
LEFT:  233
LEFT:  232
LEFT:  231
LEFT:  230
LEFT:  229
LEFT:  228
LEFT:  227
LEFT:  226
LEFT:  225